# Introduction


**This notebook is a guided machine learning project for beginners. This is one of the most important and popular applications of machine learning. This application is used by almost everyone that has a device now, and has subscriptions on streaming services like Netflix, Amazon Prime, etc. 
For this project, the user will input a movie's name and the system will recommend more similar movies to them.
There will be comments every step of the way so there is a clear understanding.**

# Types of Recommendation Systems

**Mainly, there are three types of recommendation engines; collaborative filtering, content-based filtering and a hybrid of the two.**

* **Collaborative filtering**: Focuses on collecting and analyzing data on user behavior, activities, and preferences, to predict what a person will like, based on their **similarity to other users**.
* **Content-based filtering**: Works on the principle that **if you like a particular item, you will also like this other item**. To make recommendations, algorithms use a profile of the customer’s preferences and a description of an item (genre, product type, color, word length) to work out the similarity of items. (We will use this for the project)
* **Hybrid recommendation engine**: Looks at **both** the meta (collaborative) data and the transactional (content-based) data. Because of this, it outperforms both.

# Importing libraries and dataset

In [1]:
import numpy as np
import pandas as pd
import difflib # Will compare the movies and find the most similar ones
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movie_data = pd.read_csv("../input/azmovies/movies (1).csv")
movie_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [3]:
movie_data.shape

(4803, 24)

Total 4803 rows and 24 columns in this dataset

# Feature Selection

In [4]:
# Choosing columns that we need for the system
features = ["genres","keywords","popularity","tagline","vote_average","cast","director"]

# Data Cleaning

In [5]:
# Checking for missing values
movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

**Except for popularity and vote_average columns, we have missing values in the rest of the selected columns.**

In [6]:
# Replacing null values with empty string in the feature columns
for feature in features:
    movie_data[feature] = movie_data[feature].fillna('')

In [7]:
movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4803 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4803 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

**No more missing values.**

In [8]:
# Now, let's combine all our features
data = movie_data['genres'] + movie_data['keywords'] + str(movie_data['popularity']) + movie_data['tagline'] + str(movie_data['vote_average']) + movie_data['cast'] + movie_data['director']
data

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy Romance0       150.437577\n1       139....
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    0       150.437577\n1       139.082615\n2     ...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object

# Vectorize Features

**Now, to use the cosine similarity, we will have to convert all features to feature vectors (numeric values). Text data cannot be used.**

In [9]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(data)
print(vectors)

  (0, 3257)	0.16388540179242558
  (0, 20291)	0.2567923468827883
  (0, 16596)	0.14931969761581868
  (0, 14694)	0.20974218104576342
  (0, 22571)	0.15547532948542722
  (0, 25526)	0.1841488052718978
  (0, 21868)	0.20207224730210313
  (0, 20968)	0.20207224730210313
  (0, 26432)	0.1866924881846971
  (0, 26092)	0.220169184866525
  (0, 9945)	0.1925588775788962
  (0, 25218)	0.029233911711366724
  (0, 18327)	0.24493901571031446
  (0, 17961)	0.08149259594265924
  (0, 26082)	0.11833473959491154
  (0, 23370)	0.07050023310419197
  (0, 9038)	0.24493901571031446
  (0, 6792)	0.05846782342273345
  (0, 105)	0.05846782342273345
  (0, 14964)	0.05846782342273345
  (0, 19084)	0.029233911711366724
  (0, 17472)	0.05846782342273345
  (0, 134)	0.029233911711366724
  (0, 104)	0.05846782342273345
  (0, 129)	0.029233911711366724
  :	:
  (4802, 105)	0.06173234448817456
  (4802, 14964)	0.06173234448817456
  (4802, 19084)	0.03086617224408728
  (4802, 17472)	0.06173234448817456
  (4802, 134)	0.03086617224408728
  (4802

# Cosine Similarity

In [10]:
# Now, we will find the similarity scores
similarity = cosine_similarity(vectors)
print(similarity)

[[1.         0.1041583  0.06144267 ... 0.04865892 0.05298896 0.0487263 ]
 [0.1041583  1.         0.06931347 ... 0.05551766 0.0479129  0.04405859]
 [0.06144267 0.06931347 1.         ... 0.05078047 0.11104573 0.05085079]
 ...
 [0.04865892 0.05551766 0.05078047 ... 1.         0.05587021 0.05850741]
 [0.05298896 0.0479129  0.11104573 ... 0.05587021 1.         0.05594757]
 [0.0487263  0.04405859 0.05085079 ... 0.05850741 0.05594757 1.        ]]


**Basically, a comparison of each movie with every other and finding a similarity score for each.**

# Input

In [11]:
# Taking user input
user_movie = input("Movie Name: ")

StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.

In [ ]:
# Listing all movie titles
titles = list(movie_data['title'])
# print(titles)

In [ ]:
# Finding the most similar movies to the one input by the user
similar = difflib.get_close_matches(user_movie,titles)
print(similar[0]) # we mentioned [0] because it returns a list of movies with similar titles and the 0th index gives the most similar movie to the one input by the user

In [ ]:
# Now let's find the index number of the movie we found that is most similar so we can use that index and later compare to return a list of similar movies
index_val = movie_data[movie_data.title == similar[0]]['index'].values[0]
print(index_val)

# Find similar movies

**Now, using the index value and similarity scores, we will find a list of similar movies.**

In [ ]:
score = list(enumerate(similarity[index_val])) # similarity with other movies relative to index of user_input (most similar found)
#score

In [ ]:
# Now we'll sort the movies based on their similarity score (In descending order)

sorted_scores = sorted(score, key = lambda x:x[1], reverse = True) 
#print(sorted_scores)

In [ ]:
# Finally, print the top 20 names of similar movies based on the index

print('Recommended Movies: \n')

i = 1
for movie in sorted_scores:
    index = movie[0]
    index_title = movie_data[movie_data.index==index]['title'].values[0]
    if (i<21):
        print(i, '.',index_title)
        i+=1

# Workflow


**So basically the workflow is like this: Import libraries and dataset -> Feature selection -> check for missing values -> perform necessary imputation -> Vectorize features -> Find cosine similarity -> Find similar movies.**

A larger dataset with more movies would undoubtedly return more similar movies.

A special thanks to Siddhardhan S (youtube.com/c/Siddhardhan) for his amazing tutorials.

# Movie Recommendation System

In [ ]:
user_movie = input("Movie Name: ")

titles = list(movie_data['title'])
similar = difflib.get_close_matches(user_movie,titles)
index_val = movie_data[movie_data.title == similar[0]]['index'].values[0]
score = list(enumerate(similarity[index_val]))
sorted_scores = sorted(score, key = lambda x:x[1], reverse = True) 

print('Recommended Movies: \n')

i = 1
for movie in sorted_scores:
    index = movie[0]
    index_title = movie_data[movie_data.index==index]['title'].values[0]
    if (i<21):
        print(i, '.',index_title)
        i+=1
        
# Entire code is broken down above.

You can also build this by copying all the codes and compiling them together.